# Soulmate Matching Problem

Suppose you are given $N$ people, and each person has a single soulmate (suppose $N$ is even). Each round, you get to pair two people and check if they're soulmates. Is there a way to match people to minimize the number of rounds needed to match everyone?

In [39]:
import numpy as np
from itertools import permutations
from math import factorial

def generate_matchings_helper(remaining):
    # number nodes 1 to n
    # return a list of all possible matchings
    # where each matching is a list of tuples
    
    # for example, n=4 should return
    # [[(1, 2), (3, 4)], [(1, 3), (2, 4)], [(1, 4), (2, 3)]]
    # ((1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4))
    # 2, 3, 4
    # note that the order of the tuples within the list does not matter

    if len(remaining) == 2:
        return [[(remaining[0], remaining[1])]]
    else:
        result = []
        # match first node with each other node
        for i in range(1, len(remaining)):
            # match first node with i-th node
            # and recursively call the function with the remaining nodes
            # and append the result to the list
            result += [[(remaining[0], remaining[i])] + x for x in generate_matchings_helper(remaining[1:i] + remaining[i+1:])]
        return result

def generate_matchings(n):
    return generate_matchings_helper(list(range(1, n+1)))

def double_factorial(n):
    # return the double factorial of n
    if n == 1 or n == 2:
        return n
    else:
        return n * double_factorial(n-2)

def predicted_combinations(n):
    return double_factorial(n - 1)

In [40]:
for i in range(1, 6):
    print(f"N: {2 * i}, predicted combinations: {predicted_combinations(2 * i)}, actual combinations: {len(generate_matchings(2 * i))}")

N: 2, predicted combinations: 1, actual combinations: 1
N: 4, predicted combinations: 3, actual combinations: 3
N: 6, predicted combinations: 15, actual combinations: 15
N: 8, predicted combinations: 105, actual combinations: 105
N: 10, predicted combinations: 945, actual combinations: 945


In [41]:
# a policy is an ordering of the edges
# our question: is one policy better than another?

def generate_all_edges(n):
    # return a list of all possible edges
    # for example, n=4 should return
    # [(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
    return [(i, j) for i in range(1, n+1) for j in range(i+1, n+1)]

def random_policy(n):
    # return a random permutation of all edges
    # we need to use np.randoom.permutation because generating the list of
    # all permutations, then choosing a random one, is too slow
    return np.random.permutation(generate_all_edges(n)).tolist()

def evaluate(policy, matching):
    already_matched = []
    score = 0

    for edge in policy:
        if edge[0] in already_matched or edge[1] in already_matched:
            continue
        if (edge[0], edge[1]) in matching or (edge[1], edge[0]) in matching:
            already_matched.append(edge[0])
            already_matched.append(edge[1])
        score += 1

    return score

def evaluate_policy(policy, matchings):
    # return the average score of the policy
    return sum([evaluate(policy, matching) for matching in matchings]) / len(matchings)

In [42]:
def find_best_policy(n):
    best_policies = None
    best_score = np.inf
    generated_matchings = generate_matchings(n)

    for policy in permutations(generate_all_edges(n)):
        score = evaluate_policy(policy, generated_matchings)

        if score < best_score:
            best_score = score
            best_policies = [policy]
        elif score == best_score:
            best_policies.append(policy)

    return best_policies

def find_random_best_policy(n, num_policies):
    best_policies = None
    best_score = np.inf
    generated_matchings = generate_matchings(n)

    for _ in range(num_policies):
        policy = random_policy(n)
        score = evaluate_policy(policy, generated_matchings)

        if score < best_score:
            best_score = score
            best_policies = [policy]
        elif score == best_score:
            best_policies.append(policy)

    return best_policies

In [43]:
N = 8

best_policies = find_random_best_policy(N)

print(f"Number of best policies for n=4: ", len(best_policies))
print(f"Number of total policies for n=4: ", factorial(N * (N - 1) // 2))
print(best_policies[0])
print("The above policy has the score ", evaluate_policy(best_policies[0], generate_matchings(N)))

Number of best policies for n=4:  288
Number of total policies for n=4:  720
((1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4))
The above policy has the score  3.0


In [67]:
N = 8

best_policies = find_random_best_policy(N, 10000)

print(f"Best policy found for n=6:", best_policies[0])
print("The above policy has the score ", evaluate_policy(best_policies[0], generate_matchings(N)))
print(f"Number of total policies for n=6:", factorial(N * (N - 1) // 2))

Best policy found for n=6: [[2, 5], [2, 7], [2, 3], [5, 7], [5, 6], [4, 5], [2, 4], [1, 5], [5, 8], [4, 7], [1, 2], [1, 7], [3, 4], [7, 8], [2, 8], [4, 8], [3, 8], [2, 6], [1, 8], [6, 8], [1, 4], [3, 6], [4, 6], [1, 6], [3, 7], [1, 3], [6, 7], [3, 5]]
The above policy has the score  10.628571428571428
The predicted best policy should have score 24.0
Number of total policies for n=6: 304888344611713860501504000000
